### test runexp.py

In [112]:
import pandas as pd
from croqr.common.config import LOCAL_DATA_DIR 
from croqr.ctdq.fs import MyNN, MyLogisticRegression, runselector
import os
import numpy as np

from croqr.ctdq.fs import getfeaturenames
from sklearn.linear_model import LogisticRegression,Lasso,LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import get_scorer,balanced_accuracy_score,mean_squared_error,r2_score,cohen_kappa_score, confusion_matrix,f1_score,classification_report,roc_auc_score,roc_curve,auc,recall_score,precision_score,precision_recall_curve,average_precision_score
from sklearn.model_selection import cross_val_score,RandomizedSearchCV
DF=pd.DataFrame
import xgboost as xgb

In [2]:
resample = '1Min'

In [7]:
dffilename='dfbt'+resample+'f.pkl'#'dfbt30sf.pkl' dfbt30Sfandinter.pkl
pd.set_option("display.precision", 3)

In [9]:
df=pd.read_pickle(os.path.join(LOCAL_DATA_DIR, dffilename)).ffill()

In [ ]:
list(df.columns)

In [ ]:
df.head()

In [17]:
print(f"dfcols={list(df.columns)}")
#keep only float columns,  not time
#ipdb.set_trace()
df=df.select_dtypes(include=[np.float,np.int,np.int64,np.int32])
print(f"only floats={list(df.columns)}")

dfcols=['inst_id', 'provider_id', 'event_date', 'exch_ts', 'local_ts', 'bestb', 'besta', 'amb', 'm', 'bv1', 'av1', 'vimb1', 'bc1', 'ac1', 'cimb1', 'vwap1', 'bv5', 'av5', 'vimb5', 'bc5', 'ac5', 'cimb5', 'vwap5', 'bv20', 'av20', 'vimb20', 'bc20', 'ac20', 'cimb20', 'vwap20', 'bv25', 'av25', 'vimb25', 'bc25', 'ac25', 'cimb25', 'vwap25', 'ybb', 'ybs', 'qimb', 'signimb', 'vwap', 'qb', 'qs', 'nb', 'ns', 'o', 'h', 'l', 'c', 'qbp', 'qsp', 'nbp', 'nsp', 'r', 'dpinbuy', 'dpinsell', 'dpin', 'dpins', 'hml', 'rl+1.c', 'ycb', 'ycs', 'noise1', 'noise2', 'rl-1.m', 'rl-1.amb', 'rl-1.vwap1', 'rl-1.vwap5', 'rl-1.c', 'rl-1.qimb', 'rl-1.signimb', 'rl-1.vwap', 'rl-1.qb', 'rl-1.qs', 'rl-1.qbp', 'rl-1.qsp', 'rl-1.nbp', 'rl-1.nsp', 'rl-1.dpinbuy', 'rl-1.dpinsell', 'rl-1.dpin', 'rl-1.dpins', 'rl-1.hml', 'ra-1.vimb1', 'ra-1.cimb1', 'ra-1.cimb5', 'ra-1.vimb20', 'ra-1.signimb', 'ra-1.r', 'qrandn2,15.rl-1.m', 'qrandn2,15.rl-1.amb', 'qrandn2,15.rl-1.vwap1', 'qrandn2,15.rl-1.vwap5', 'qrandn2,15.rl-1.c', 'qrandn2,15.rl

In [27]:
yrs=getfeaturenames('y', df.columns)

In [28]:
yrs=[yr for yr in yrs if df[yr].nunique()>10] # yr stand for?
print(f"yrs={yrs}")

yrs=['rl+1.c']


In [23]:
df.columns

Index(['inst_id', 'provider_id', 'exch_ts', 'local_ts', 'bestb', 'besta',
       'amb', 'm', 'bv1', 'av1',
       ...
       'qrandn5,100.rl-1.dpins', 'qrandn5,100.rl-1.hml',
       'qrandn5,100.ra-1.vimb1', 'qrandn5,100.ra-1.cimb1',
       'qrandn5,100.ra-1.cimb5', 'qrandn5,100.ra-1.vimb20',
       'qrandn5,100.ra-1.signimb', 'qrandn5,100.ra-1.r', 'qrandn5,100.noise1',
       'qrandn5,100.noise2'],
      dtype='object', length=141)

In [54]:
yrs

['rl+1.c']

In [60]:
xsraw = ['m', 'amb', 'nbp', 'dpinsell', 'qbp', 'dpin', 'qsp', 'qimb', 'signimb', 'hml', 'vimb1',
                                'vimb5', 'cimb1', 'cimb5', 'vwap1',
                                'r']
xs = 'all'

In [103]:
xsraw = getfeaturenames('raw', df.columns, xsraw=xsraw)
xsq3 = getfeaturenames('q3', df.columns, xsraw=xsraw)
xsq95 = getfeaturenames('q95', df.columns, xsraw=xsraw)

xsall = getfeaturenames('all', df.columns)
xsinteract = getfeaturenames('interact', df.columns)
y = 'ycb'

models = ["log", "lin", "xgb", "nn", "plog", "plin", "pn",'xgbc','xgbrf']

In [62]:
xs = {'all': xsall, 'raw': xsraw, 'q3': xsq3, 'q95': xsq95}[xs]


In [63]:
df.count(1).value_counts()

141    100
dtype: int64

In [64]:
optiter = 1
cv = 2
df = df.iloc[:200].dropna()
dftrain = df.iloc[:int(0.8 * len(df))]
dftest = df.iloc[-int(0.1 * len(df)):]

In [65]:
xsraw

['m',
 'dpin',
 'qimb',
 'amb',
 'cimb1',
 'dpinsell',
 'signimb',
 'cimb5',
 'hml',
 'vimb5',
 'r',
 'vimb1',
 'qbp',
 'qsp',
 'nbp',
 'vwap1']

In [ ]:
dftrainxsraw

In [86]:
for yr in yrs:
    olstrain = dftrain[xsraw].ffill()
    olstest = dftest[xsraw].ffill()

    ols = LinearRegression()
    scaler = StandardScaler()
    scaler.fit(olstrain)
    ols.fit(scaler.transform(olstrain), dftrain[yr])
    r2is = ols.score(scaler.transform(olstrain), dftrain[yr])
    r2os = ols.score(scaler.transform(olstest), dftest[yr])
    #    ipdb.set_trace()

    if yr == 'rl+1.c' and y == 'ycb':
        ypis = (ols.predict(scaler.transform(olstrain)) > 0).astype(int)
        ypos = (ols.predict(scaler.transform(olstest)) > 0).astype(int)
        f1is = f1_score(dftrain[y], ypis)
        f1os = f1_score(dftest[y], ypos)


    if yr == 'rl+1.c' and y == 'ycs':
        ypis = (ols.predict(scaler.transform(olstrain)) < 0).astype(int)
        ypos = (ols.predict(scaler.transform(olstest)) < 0).astype(int)
        f1is = f1_score(dftrain[y], ypis)
        f1os = f1_score(dftest[y], ypos)

print(f1is, f1os)
    # among the metric, what metrix do you need to use? 

0.6666666666666667 0.3333333333333333


In [85]:
f1is, f1os

(0.6666666666666667, 0.3333333333333333)

In [96]:
n_iter = optiter

if 'nn' in models or 'pnn' in models:
    fixedparams = dict(scale_pos_weight=10, output_bias=0.9)
    params = {'nhidden1': 5, 'nfirst': 10, 'epochs': 30, 'dropout': 0.2, 'batch_size': 10}
    model = MyNN(**fixedparams, **params)
    params = dict(
        lr=[0.005, 0.1],
        epochs=[100, 200],
        batch_size=[10, 100],
        nfirst=[10, 20, 100],
        nhidden1=[1, 5, 20],
        nhidden2=[0, 1, 5, 10],
        dropout=[0.2, 0])
    randcv = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=n_iter, scoring='f1', n_jobs=1,
                                cv=cv, verbose=1, random_state=1).fit(dftrain[xs], dftrain[y])
    print(
        f"rscv {model.__class__.__name__} fixedparams={fixedparams} bestscore={randcv.best_score_} bestparams={randcv.best_params_} \n{DF(randcv.cv_results_).sort_values(by='mean_test_score', ascending=False)[['mean_test_score', 'std_test_score', 'params']]}")
    print(f"rscv \n{DF(randcv.cv_results_).sort_values(by='mean_test_score', ascending=False)}")
    nn = MyNN(**fixedparams, **randcv.best_params_)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


C:\Users\Wang\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



C:\Users\Wang\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



C:\Users\Wang\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



rscv MyNN fixedparams={'scale_pos_weight': 10, 'output_bias': 0.9} bestscore=0.7199180747567844 bestparams={'nhidden2': 1, 'nhidden1': 1, 'nfirst': 10, 'lr': 0.1, 'epochs': 100, 'dropout': 0.2, 'batch_size': 10} 
   mean_test_score  std_test_score  \
0 0.7199180748     0.0102406554      

                                                                                                     params  
0  {'nhidden2': 1, 'nhidden1': 1, 'nfirst': 10, 'lr': 0.1, 'epochs': 100, 'dropout': 0.2, 'batch_size': 10}  
rscv 
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0 1.254441857    0.03261053562 0.06224167347    0.001146674156    

  param_nhidden2 param_nhidden1 param_nfirst param_lr param_epochs  \
0  1              1              10          0.1       100           

  param_dropout param_batch_size  \
0 0.2            10                

                                                                                                     params  \
0  {'nhidden2': 1, 'nhid

In [102]:
if 'xgbc' in models or 'pxgbc' in models:  # False:

    params = {'scale_pos_weight': 100, 'n_estimators': 30, 'max_depth': 5, 'max_delta_step': 10, 'learning_rate': 0.1,
              'colsample_bytree': 0.8, 'base_score': 0.1, 'alpha': 1}
    fixedparams = dict(objective='binary:logistic')
    model = xgb.XGBClassifier(**fixedparams, **params)
    params = {
        'n_estimators': [10, 100, 200],
        'colsample_bytree': [0.8, 1.0],
        'max_depth': [5, 10],
        'learning_rate': [0.01, 0.1, 1],
        'alpha': [1, 10, 100],
        'scale_pos_weight': [1, 10, 100],
        'base_score': [0.1, 0.9],
        'max_delta_step': [0, 1, 10]
    }
    randcv = RandomizedSearchCV(model, param_distributions=params, n_iter=n_iter, scoring='f1', n_jobs=1, cv=cv,
                                verbose=0, random_state=1).fit(dftrain[xs], dftrain[y])
    print(
        f"rscv {model.__class__.__name__} fixedparams={fixedparams} bestscore={randcv.best_score_} bestparams={randcv.best_params_} \n{DF(randcv.cv_results_).sort_values(by='mean_test_score', ascending=False)[['mean_test_score', 'std_test_score', 'params']]}")
    print(f"rscv \n{DF(randcv.cv_results_).sort_values(by='mean_test_score', ascending=False)}")
    xgbc = xgb.XGBClassifier(**fixedparams, **randcv.best_params_)

C:\Users\Wang\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

C:\Users\Wang\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



[13:51:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:51:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:51:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
rscv XGBClassifier fixedparams={'objective': 'binary:logistic'} bestsc

C:\Users\Wang\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].



In [105]:
if 'xgbrf' in models or 'pxgbrf' in models:
    params = {'scale_pos_weight': 100, 'n_estimators': 30, 'max_depth': 5, 'max_delta_step': 10, 'learning_rate': 0.1,
              'colsample_bytree': 0.8, 'base_score': 0.1, 'alpha': 1}
    fixedparams = dict(objective='binary:logistic')
    model = xgb.XGBRFClassifier(**fixedparams, **params)
    params = {
        'n_estimators': [10, 100, 200],
        'colsample_bytree': [0.8, 1.0],
        'max_depth': [5, 10],
        'learning_rate': [0.01, 0.1, 1],
        'alpha': [1, 10, 100],
        'scale_pos_weight': [1, 10, 100],
        'base_score': [0.1, 0.9],
        'max_delta_step': [0, 1, 10]
    }
    randcv = RandomizedSearchCV(model, param_distributions=params, n_iter=n_iter, scoring='f1', n_jobs=1, cv=cv,
                                verbose=0, random_state=1).fit(dftrain[xs], dftrain[y])
    print(
        f"rscv {model.__class__.__name__} fixedparams={fixedparams} bestscore={randcv.best_score_} bestparams={randcv.best_params_} \n{DF(randcv.cv_results_).sort_values(by='mean_test_score', ascending=False)[['mean_test_score', 'std_test_score', 'params']]}")
    print(f"rscv \n{DF(randcv.cv_results_).sort_values(by='mean_test_score', ascending=False)}")
    xgbrf = xgb.XGBRFClassifier(**fixedparams, **randcv.best_params_)

[13:52:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:52:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:52:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
rscv XGBRFClassifier fixedparams={'objective': 'binary:logistic'} best

C:\Users\Wang\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

C:\Users\Wang\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

C:\Users\Wang\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the f

In [109]:
if 'log' in models or 'plog' in models:  # False:
    params = dict(tol=0.0001, C=1.0, solver='saga', max_iter=300)
    fixedparams = dict(penalty='l2', l1_ratio=None, class_weight='balanced', verbose=0)
    model = MyLogisticRegression(**fixedparams, **params)  # solver='lbfgs'
    # model=Pipeline([("pipe0",StandardScaler()),("pipe1",modellogistic)])
    # for pipelione need prefix pipe1!
    params = {
        'tol': [0.0001, 0.001, 0.01],
        'C': [0.1, 1, 10],
        'max_iter': [10, 50, 100, 300],
        'solver': ['lbfgs', 'sag', 'saga']
    }

    randcv = RandomizedSearchCV(model, param_distributions=params, n_iter=n_iter, scoring='f1', n_jobs=1, cv=cv,
                                verbose=0, random_state=1).fit(dftrain[xs], dftrain[y])
    print(
        f"rscv {model.__class__.__name__} fixedparams={fixedparams} bestscore={randcv.best_score_} bestparams={randcv.best_params_} \n{DF(randcv.cv_results_).sort_values(by='mean_test_score', ascending=False)[['mean_test_score', 'std_test_score', 'params']]}")
    print(f"rscv \n{DF(randcv.cv_results_).sort_values(by='mean_test_score', ascending=False)}")
    log = MyLogisticRegression(**fixedparams, **randcv.best_params_)  # solver='lbfgs'


rscv MyLogisticRegression fixedparams={'penalty': 'l2', 'l1_ratio': None, 'class_weight': 'balanced', 'verbose': 0} bestscore=0.3548387096774194 bestparams={'tol': 0.001, 'solver': 'lbfgs', 'max_iter': 10, 'C': 1} 
   mean_test_score  std_test_score  \
0 0.3548387097     0.3548387097      

                                                      params  
0  {'tol': 0.001, 'solver': 'lbfgs', 'max_iter': 10, 'C': 1}  
rscv 
   mean_fit_time    std_fit_time  mean_score_time  std_score_time param_tol  \
0 0.005003452301 0.0009815692902 0.002494692802   0.0004979372025 0.001       

  param_solver param_max_iter param_C  \
0  lbfgs        10             1        

                                                      params  \
0  {'tol': 0.001, 'solver': 'lbfgs', 'max_iter': 10, 'C': 1}   

   split0_test_score  split1_test_score  mean_test_score  std_test_score  \
0 0.7096774194       0                  0.3548387097     0.3548387097      

   rank_test_score  
0  1                


In [110]:
def getmodel(modelname):
    if modelname[0] == 'p':
        return Pipeline([("pipe0", StandardScaler()), ("pipe1", eval(modelname[1:]))])
    else:
        return eval(modelname)

In [ ]:
for mstr in models:
    runselector(dftrain, y=y, xs=xs, model=getmodel(mstr), nansy='.fillna(0)', nansx=None, verbose=2,
                methods=['sfsb', 'sfsf', 'rfe', 'abscoef'], dftest=dftest, scoring='f1', eval_metric='f1', cv=cv)  # ,
